Import necessary libraries: boto3, json, time, paramiko (for file transfer) for monitoring and alerting integration.

In [ ]:
import boto3
import json
import time
import paramiko

Configure AWS credentials securely.
Define functions for:


*   Creating EC2 instances with specified configurations
*   Transferring files to EC2 instances using paramiko


In [ ]:
# Securely configure AWS credentials (replace with your credentials)
session = boto3.Session(profile_name='your-profile-name')
ec2_client = session.client('ec2')

# Function to create EC2 instances
def create_ec2_instances(instance_type, ami_id, security_group_ids, key_name, count):
    instances = ec2_client.run_instances(
        ImageId=ami_id,
        InstanceType=instance_type,
        SecurityGroupIds=security_group_ids,
        KeyName=key_name,
        MinCount=count,
        MaxCount=count
    )
    return instances['Instances']

# Function to transfer files to EC2 instances
def transfer_files(instance_ips, files_to_transfer, destination_path):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    for instance_ip in instance_ips:
        ssh.connect(instance_ip, username='ec2-user', key_filename='path/to/your/key.pem')
        sftp = ssh.open_sftp()
        for file in files_to_transfer:
            sftp.put(file, f'{destination_path}/{file}')
        sftp.close()
        ssh.close()


Triggering Ansible playbook using the ansible module Integrating with Docker for container deployment using the docker module

In [ ]:
# Function to trigger Ansible playbook
def run_ansible_playbook(inventory_file, playbook_file):
    import ansible.runner
    runner = ansible.runner.Runner(
        module_name='shell',  # Adjust if using a different module
        module_args='/path/to/your/script.sh',  # Replace with your script
        pattern='all',
        forks=10,
        inventory=inventory_file
    )
    data = runner.run()
    return data

In [ ]:
# Function to create Docker containers
def create_docker_containers(image_name, ports, volumes, environment_variables):
    # Implement using Docker SDK for Python or other libraries
    # Example using docker-py library:
    import docker
    client = docker.from_env()
    container = client.containers.run(
        image_name,
        ports=ports,
        volumes=volumes,
        environment=environment_variables,
        detach=True
    )
    return container

Setting up monitoring and alerting using AWS CloudWatch

In [ ]:
# Function to set up monitoring and alerting (example using CloudWatch)
def setup_monitoring_and_alerting(instances, metrics, thresholds, alert_channel):
    # Implement using boto3 CloudWatch or third-party tools
    # Example using CloudWatch:
    cloudwatch_client = session.client('cloudwatch')
    for instance in instances:
        for metric in metrics:
            alarm = cloudwatch_client.put_metric_alarm(
                AlarmName=f'{instance["InstanceId"]}_{metric}',
                Namespace='AWS/EC2',
                MetricName=metric,
                Dimensions=[{'Name': 'InstanceId', 'Value': instance['InstanceId']}],
                Statistic='Average',
                Period=300,
                EvaluationPeriods=2,
                Threshold=thresholds[metric]['critical'],
                ComparisonOperator='GreaterThanOrEqualToThreshold',
                AlarmActions=[alert_channel],
            )

In [ ]:
# Main script
if __name__ == '__main__':
    # Replace with your specific values
    instance_type = 't2.micro'
    ami_id = 'ami-xxxxxxxx'
    security_group_ids = ['sg-xxxxxxxx']
    key_name = 'your-key-pair'
    count = 2

    instances = create_ec2_instances(instance_type, ami_id, security_group_ids, key_name, count)
    instance_ips = [instance['PublicIpAddress'] for instance in instances]

    files_to_transfer = ['file1.txt', 'file2.yaml']
    destination_path = '/home/ec2-user/'
    transfer_files(instance_ips, files_to_transfer)